In [1]:
# pip install -U pandas, plotly, unidecode, spotipy
### spotipy only needed if SEARCH_JSON is not present and is to be recreated
import os, json
from unidecode import unidecode
import plotly.express as px
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
PIANO_SOLO_TSV = '220214_debussy_piano_solo.tsv'
SEARCH_JSON = 'spotify_search_results.json'
RESULT_JSON = 'spotify_median_durations.json'

In [2]:
def make_keys(lesure_col, mvt_col):
    lesure_format =  '{:03d}'.format
    mvt_format =  '{:02d}'.format
    lesure_notna = lesure_col.notna()
    mvt_notna = mvt_col.notna()
    lesure_col = lesure_col.apply(lesure_format)
    mvt_col = mvt_col.apply(mvt_format).where(mvt_notna, pd.NA)
    key = lesure_col + ('-' + mvt_col.astype('string')).fillna('')
    key = key.where(lesure_notna, pd.NA)
    return key

short_keys = {
    "les soirs illumines par l'ardeur du charbon": "soirs",
    "la boite a joujoux": "boite",
    "etude retrouvee": "etude"}
    
pieces = pd.read_csv(PIANO_SOLO_TSV, sep='\t')
pieces = pieces[pieces.title.notna()]
lesure_regex = r"^L ?(?P<lesure>\d{1,3})(?:\/(?P<mvt>\d{1,2}))?$"
pieces = pd.concat([pieces.L.str.extract(lesure_regex, expand=True).astype('Int64'), pieces.drop(columns='L')], axis=1)
key_col = make_keys(pieces.lesure, pieces.mvt)
key_col.fillna(pieces.title.str.lower().apply(unidecode), inplace=True)
key_col.replace(short_keys, inplace=True)
pieces.insert(2, 'key', key_col)
pieces.head(10)

,lesure,mvt,key,year,title,availability,comments,pdf,source
0,9,<NA>,009,1880,Danse bohémienne,True,to check,NaN,https://musescore.com/yazen23434/danse-boh-mie...
1,<NA>,<NA>,intermede,1880,Intermède,False,NaN,NaN,NaN
2,66,1,066-01,1888,Arabesque I: Andantino con moto in E Major,True,NaN,https://imslp.org/wiki/Special:ReverseLookup/2821,https://musescore.com/hmscomp/debussy-premiere...
3,67,<NA>,067,1890,Mazurka,True,several errors,NaN,https://musescore.com/user/4887176/scores/5699433
4,68,<NA>,068,1890,Rêverie,True,NaN,NaN,https://musescore.com/hmscomp/r-verie-claude-d...
5,69,<NA>,069,1890,Tarentelle styrienne,True,NaN,NaN,https://musescore.com/user/56409/scores/263276...
6,70,<NA>,070,1890,Ballade slave,True,NaN,NaN,https://musescore.com/user/3797871/scores/5825949
7,71,<NA>,071,1890,Valse romantique,True,few notes to fix,NaN,https://musescore.com/user/4887176/scores/5696...
8,75,1,075-01,1890-1905,Prélude,True,NaN,NaN,https://musescore.com/user/85337/scores/615026...
9,75,2,075-02,1890-1905,Menuet,True,few notes to fix(free version),NaN,https://musescore.com/user/85337/scores/614817...


In [3]:
if os.path.isfile(SEARCH_JSON):
    with open(SEARCH_JSON, 'r', encoding='utf-8') as f:
        searches = json.load(f)
        print("Loaded search results from " + SEARCH_JSON)
else:
    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials
    client_credentials_manager = SpotifyClientCredentials('---','---')
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    searches = {}
    for key, lesure, title in pieces[['key', 'lesure', 'title']].itertuples(index=False):
        query = f"debussy {unidecode(title)}"
        if not pd.isnull(lesure):
            query += f" {lesure}"
        print(f"{key}: query string '{query}'")
        searches[key] = sp.search(query, limit=50, type='track')
    with open(SEARCH_JSON, 'w', encoding='utf-8') as f:
        json.dump(searches, f)
        print("Search results stored as " + SEARCH_JSON)

Loaded search results from spotify_search_results.json


In [4]:
less_than_five = {key: len(search['tracks']['items']) for key, search in searches.items() if len(search['tracks']['items']) < 5}
print(f"These keys have less than five results: {less_than_five}.\n\nExample search result:")
{k: type(v) if type(v) in (dict, list) else v for k, v in searches['123-09']['tracks']['items'][0].items()}

These keys have less than five results: {'123-09': 2}.

Example search result:


{'album': dict,
 'artists': list,
 'available_markets': list,
 'disc_number': 2,
 'duration_ms': 148106,
 'explicit': False,
 'external_ids': dict,
 'external_urls': dict,
 'href': 'https://api.spotify.com/v1/tracks/2NB4I6PW0wyX43foAITpO3',
 'id': '2NB4I6PW0wyX43foAITpO3',
 'is_local': False,
 'name': 'Preludes, Book 2, L 123 - Hommage À Samuel Pickwick Esquire',
 'popularity': 0,
 'preview_url': 'https://p.scdn.co/mp3-preview/9e8e64dc4903d670d7f8c16a6aa298a4000eabb4?cid=5ee5f9cba29f454fabd7bb5316eff69e',
 'track_number': 24,
 'type': 'track',
 'uri': 'spotify:track:2NB4I6PW0wyX43foAITpO3'}

### Obtaining dict structure 
lesure -> [{artist, id, duration_ms, release_date}]

In [5]:
pieces_data = {}
for key, search_results in searches.items():
    tracks = search_results['tracks']['items']
    if not tracks: print(f'{"missing tracks for "}{key}')
    search_tracks = []    
    already_found = []
    for (idx, track) in enumerate(tracks):
        artists = [artist['name'] for artist in track['artists'] if 'debussy' not in artist['name'].lower()]
        n_artists = len(artists)
        if n_artists == 0:
            print(f"{key}: track[{idx}] doesn't come with artist's name, skipping")
            continue
        if n_artists > 1:
            print(f"{key}: track[{idx}] has more than 1 artist: {artists}, skipping")
            continue
        artist = artists[0]
        duration_ms = track['duration_ms']
        id_tuple = (artist, duration_ms)
        if id_tuple in already_found:
            continue
        already_found.append(id_tuple)
        name = track['name']
        id = track['id']
        release_date = track['album']['release_date']        
        
        track_info = {
            'duration_ms' : duration_ms, 
            'name' : name,
            'artist' : artist,
            'url' : 'https://open.spotify.com/track/' + id,
            'release_date' : release_date  
        }
        search_tracks.append(track_info)
    pieces_data[key] = search_tracks
pieces_data['123-09']

intermede: track[0] has more than 1 artist: ['Michael Barenboim', 'Daniel Barenboim'], skipping
intermede: track[1] has more than 1 artist: ['Rafał Blechacz', 'Bomsori'], skipping
intermede: track[2] has more than 1 artist: ['Michael Barenboim', 'Daniel Barenboim'], skipping
intermede: track[3] has more than 1 artist: ['Michael Barenboim', 'Daniel Barenboim'], skipping
intermede: track[4] has more than 1 artist: ['David Oistrakh', 'Frida Bauer'], skipping
intermede: track[5] has more than 1 artist: ['Rafal Blechacz', 'Bomsori'], skipping
intermede: track[6] has more than 1 artist: ['David Oistrakh', 'Frida Bauer'], skipping
intermede: track[7] has more than 1 artist: ['Augustin Dumay', 'Maria João Pires'], skipping
intermede: track[8] has more than 1 artist: ['Rafał Blechacz', 'Bomsori'], skipping
intermede: track[9] has more than 1 artist: ['Augustin Dumay', 'Maria João Pires'], skipping
intermede: track[10] has more than 1 artist: ['Martha Argerich', 'Dora Schwarzberg'], skipping
int

[{'duration_ms': 154932,
  'name': 'Préludes, Livre II, L. 123: No. 9, Hommage à Samuel Pickwick esq. P.P.M.P.C.',
  'artist': 'Jérome Granjon',
  'url': 'https://open.spotify.com/track/4ag3EZtqO8MiMWlo5vKIKQ',
  'release_date': '2011'}]

### Output track names to exclude wrong ones

In [6]:
track_titles = {}
for key, title in pieces.loc[pieces.key.notna(), ['key', 'title']].itertuples(index=False):
    recordings = pieces_data[key]
    track_info = [(track['name'], track['artist']) for track in recordings]
    track_titles[(key, title)] = list(enumerate(track_info))
track_titles

{('009',
  'Danse bohémienne'): [(0,
   ('Danse bohémienne, L.9', 'Gordon Fergus-Thompson')), (1,
   ('Danse bohémienne, L. 9', 'Werner Haas')), (2,
   ('Danse bohémienne, L. 9, CD 4', 'Jean-Efflam Bavouzet')), (3,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Aldo Ciccolini')), (4,
   ('Danse Bohémienne, L.9', 'Jean-Yves Thibaudet')), (5,
   ('Danse bohémienne, L. 9', 'Bruno Canino')), (6,
   ('Danse Bohémienne, L. 9', 'John Paul Jalwan')), (7,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Walter Gieseking')), (8,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Walter Gieseking')), (9,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Aldo Ciccolini')), (10,
   ('Debussy: Danse bohémienne, CD 4, L. 9', 'Monique Haas')), (11,
   ('Claude Debussy: Danse bohémienne, L 9 (1880)',
    'Jean-Pierre Armengaud')), (12,
   ('Danse Bohémienne, L.9', 'Jean-Yves Thibaudet')), (13,
   ('Danse bohémienne, L.9', 'Jean-Louis Haguenauer')), (14,
   ('Danse Bohémienne, L. 9', 'Daniel Ericourt')), (15,
   ('D

#### Exclude wrong search results and store
If more erroneous results are to be found, include them in the following cell.

In [7]:
pieces_data['095-01'] = pieces_data['095-01'][:21] + pieces_data['095-01'][23:] # results #21 and #22 contained movements II & III

### Inspect data to exclude outliers

In [8]:
duration_series = {key: pd.Series([track['duration_ms'] for track in tracks], dtype=float) / 1000 for key, tracks in pieces_data.items()}
seconds = pd.concat(duration_series.values(), keys=duration_series.keys(), axis=1).sort_index(axis=1)
stats = seconds.describe()
stats

,009,066-01,066-02,067,068,069,070,071,075-01,075-02,075-03,075-04,082,087-01,087-02,087-03,095-01,095-02,095-03,099,100-01,100-02,100-03,105,106,108,110-01,110-02,110-03,111-01,111-02,111-03,113-01,113-02,113-03,113-04,113-05,113-06,114,115,117-01,117-02,117-03,117-04,117-05,117-06,117-07,117-08,117-09,117-10,117-11,117-12,121,123-01,123-02,123-03,123-04,123-05,123-06,123-07,123-08,123-09,123-10,123-11,123-12,131-01,131-02,131-03,131-04,131-05,131-06,132,133,136-01,136-02,136-03,136-04,136-05,136-06,136-07,136-08,136-09,136-10,136-11,136-12,138,boite,etude,intermede,soirs
count,22.000000,12.00000,4.000000,22.000000,22.000000,5.000000,33.000000,25.000000,32.000000,29.000000,20.000000,23.000000,20.00000,16.000000,15.000000,31.000000,27.000000,24.000000,31.000000,30.000000,23.000000,19.000000,15.00000,37.000000,32.000000,11.000000,42.000000,25.000000,27.000000,29.000000,22.000000,38.000000,19.000000,30.000000,18.000000,18.000000,15.000000,14.000000,22.000000,6.000000,23.000000,31.000000,21.000000,26.000000,31.000000,24.000000,27.000000,27.000000,15.000000,30.000000,28.000000,18.000000,18.000000,22.000000,16.000000,17.000000,24.000000,16.000000,8.00000,22.000000,24.000000,1.000,22.000000,15.000000,32.000000,4.000000,0.0,2.000000,1.000,9.000000,1.000,22.000000,12.000000,11.000000,17.000000,16.000000,15.000000,9.000000,11.000000,15.000000,19.000000,13.000000,11.000000,17.000000,18.000000,7.000000,1.000,10.000000,8.000000,22.000000
mean,125.128318,260.85150,204.859750,172.687682,270.185364,329.367000,388.577212,222.542680,261.927375,246.362724,313.629850,221.013391,406.50230,229.839188,296.192200,242.888323,246.798037,322.394292,236.390161,293.953800,298.355304,319.813211,226.17780,286.932811,346.935219,51.026273,309.517976,438.337160,212.010370,278.062310,331.849318,246.779474,135.938684,207.334067,161.576611,153.813111,156.585333,171.770429,107.623955,141.332500,199.301348,235.168968,130.555524,224.867000,186.047871,258.692708,209.262593,159.509852,164.052667,390.341333,166.925321,152.982278,275.040667,185.124864,211.990187,205.495882,186.459875,182.705938,150.73825,286.132682,194.465583,154.932,178.523045,159.108667,266.083781,149.919750,NaN,148.613000,144.866,193.230889,138.386,262.195818,72.855167,180.911545,225.835176,303.186813,246.983733,228.305222,92.872818,135.269667,292.260789,191.357231,312.834455,262.455588,286.021944,137.926857,203.906,274.415500,249.554250,144.714227
std,9.124795,52.40571,22.399967,18.585694,38.569933,15.080555,37.283674,31.154354,44.152324,34.793468,37.095403,19.706413,45.88300,25.880611,24.746577,20.337853,20.426549,52.664422,15.305768,23.365306,19.519497,18.964736,30.97971,30.602937,60.765903,8.525765,23.733621,51.550095,21.472259,20.002946,26.762725,23.860415,14.036213,25.796917,21.878847,22.161676,30.231483,12.317841,42.776724,20.286227,27.825262,41.325497,8.303414,26.284175,20.071132,25.423868,20.483261,20.273003,23.921561,32.817267,19.367737,33.234706,44.162385,21.658558,34.720571,23.939415,18.208729,25.261967,13.36626,37.901902,26.807329,NaN,26.271596,19.778265,47.185662,18.844722,NaN,10.634886,NaN,21.764710,NaN,23.078876,15.577426,12.491417,26.815092,35.363376,25.938990,189.984368,8.368840,7.425696,23.055771,11.300700,26.999480,43.152719,29.617924,12.817831,NaN,28.035502,21.138566,19.149001
min,111.843000,223.06600,171.600000,137.570000,214.386000,312.920000,289.529000,164.453000,215.583000,141.176000,259.665000,174.000000,346.33300,202.000000,259.680000,205.053000,203.520000,250.466000,217.306000,242.733000,266.133000,294.173000,175.00000,223.188000,94.750000,39.373000,245.160000,374.013000,180.803000,252.000000,294.000000,193.148000,117.186000,153.200000,118.077000,93.826000,119.160000,144.520000,81.693000,108.221000,141.333000,163.333000,114.666000,179.840000,154.493000,212.160000,173.016000,137.840000,133.061000,330.880000,130.626000,129.933000,234.106000,145.840000,145.213000,157.080000,150.373000,140.066000,127.25300,214.653000,142.413000,154.932,135.266000,116.586000,199.239000,132.373

In [9]:
px.box(seconds, title="Recording times in seconds; whiskers/outliers: +/- 1.5 * inter-quartile range")

In [10]:
def filter_outliers(S, iqr_factor=1.5):
    q1, q3 = S.quantile(.25), S.quantile(.75)
    iqr = q3 - q1
    lower, upper = q1 - iqr_factor * iqr, q3 + iqr_factor * iqr
    return S[S.between(lower, upper)]

def without_outliers(df, iqr_factor=1.5):
    filtered = df.apply(filter_outliers, iqr_factor=iqr_factor)
    difference = filtered.isna().sum() - df.isna().sum()
    print(f"Number of excluded recordings based on {iqr_factor}*IQR")
    print(difference[difference > 0])
    return filtered

filtered = without_outliers(seconds, 3)
with open(RESULT_JSON, "w", encoding='utf-8') as f:
    json.dump(filtered.median().to_dict(), f)    
px.box(filtered)

Number of excluded recordings based on 3*IQR
066-01       1
070          1
075-01       1
087-02       1
105          1
106          2
110-03       1
111-03       1
113-04       1
114          1
117-05       1
117-09       1
117-12       1
123-04       1
123-05       1
123-12       1
133          1
136-05       1
136-10       2
intermede    1
dtype: int64


In [11]:
def check(key, k=3):
    """k=3: show 3 longest; k=-3: show 3 shortest"""
    S = seconds[key]
    if k < 0:
        ascending = True
        k = abs(k)
    else:
        ascending = False
    S = S[S.notna()].sort_values(ascending=ascending).iloc[:k]
    for i in S.index:
        print(pieces_data[key][i])

check('110-02', 3)

{'duration_ms': 568533, 'name': 'Images, (Prèmiere Série), L. 110: II. Hommage à Rameau', 'artist': 'Axel Gillison', 'url': 'https://open.spotify.com/track/7Ed0j4GWZ7Ug0PSRZR9yyy', 'release_date': '2022-02-12'}
{'duration_ms': 555880, 'name': 'Images, Book 1, L. 110: No. 2, Hommage à Rameau (Remastered 2021) [Live]', 'artist': 'Sergio Fiorentino', 'url': 'https://open.spotify.com/track/412CsxBOEVYSiOX0afXCry', 'release_date': '2021-12-10'}
{'duration_ms': 514106, 'name': 'Debussy: Images, Livre I, CD 105, L. 110: No. 2, Hommage à Rameau', 'artist': 'Aldo Ciccolini', 'url': 'https://open.spotify.com/track/14RaxqygjRIlDR4u1bH5k6', 'release_date': '2019-08-30'}
